<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/3d1d_FEMdG_nosolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2025-12-04 16:56:27--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-12-04 16:56:28 (74.9 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f62c6f6/releases/pybi

In [8]:
from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title 3D-FEM,1D-dG time-dependent advection-diffusion

def FEM3D_dG1D(kappa, mesh_size, T):
  mesh_vec = mesh_size
  kappa    = kappa
  T        = T

  for i in mesh_vec:
    ncell   = 4*(2**i)
    mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
    radius  = 0.05
    dt      = 0.1#/ncell
    inv_dt  = Constant(1.0/dt)
    tpts    = int(T/dt)

    # === Perimeter and Area ===
    D_perimeter = 2*np.pi*radius
    D_area = np.pi*(radius**2)

    # === Create Meshes ===
    Omega = MeshFunction("size_t",mesh_3d,1,0)
    # CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1) #straight line
    CompiledSubDomain("near(x[0]-x[1], 0.0) && near(x[0]-x[2], 0.0)").mark(Omega, 1) #diagonal line
    Lambda = EmbeddedMesh(Omega,1)
    Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
    CompiledSubDomain("near(x[0], -0.5) && near(x[1], -0.5) && near(x[2], -0.5)").mark(Lambda_boundaries, 1)
    CompiledSubDomain("near(x[0],  0.5) && near(x[1],  0.5) && near(x[2],  0.5)").mark(Lambda_boundaries, 2)

    # === Function Spaces ===
    V3 = FunctionSpace(mesh_3d, "CG", 1)
    V1 = FunctionSpace(Lambda, "DG", 1)
    W = [V3, V1]
    u3, u1 = map(TrialFunction, W)
    v3, v1 = map(TestFunction, W)

    # === Velocity terms ===
    q1 = Constant(1.0)
    # q3 = as_vector((0,0,q1)) # if straight line
    sqrt3 = np.sqrt(3.0)
    tau = as_vector((1.0/sqrt3, 1.0/sqrt3, 1.0/sqrt3))  # unit tangent along the diagonal
    q3 = q1 * tau # diagonal line

    # === Initial concentration ===
    cinlet = Constant(5.0)
    #cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


    # === Mesh Measures ===
    dxOmega = Measure("dx", domain=mesh_3d)
    dxLambda = Measure("dx", domain=Lambda)
    dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
    dSLambda = Measure("dS", domain=Lambda)
    n = FacetNormal(Lambda)
    h = CellDiameter(Lambda)
    alpha = Constant(50.0)
    epsilon = Constant(-1.0)
    perf1 = 1.0
    perf3 = 1.0

    # === Upwind ===
    u1_up = conditional(ge(q1, 0.0), u1('-'), u1('+'))  # upstream trace


    # === Cylinder ===
    cylinder = Circle(radius=radius, degree=20)

    # === Averages ===
    u3_avg = Average(u3, Lambda, cylinder)
    v3_avg = Average(v3, Lambda, cylinder)

    # === Initialize time step ===
    uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
    uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

    #r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

    # === Begin time-stepping ===
    t = 0.0
    for step in range(tpts):
      t += dt
      print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

      #cinlet.t = t # include again if cinlet is time-varying


      # === Variational Forms ===
      a00 = inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega \
            + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda - inner(q3, grad(v3)) * u3 * dxOmega
      a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
      a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
      a11 = D_area * inv_dt * u1 * v1 * dxLambda \
          + D_area * inner(grad(u1), grad(v1)) * dxLambda \
          - D_area * inner(avg(grad(u1)), jump(v1, n)) * dSLambda \
          - D_area * inner(avg(grad(v1)), jump(u1, n)) * dSLambda \
          + (alpha/avg(h)) * jump(u1) * jump(v1) * dSLambda \
          + kappa * D_perimeter * u1 * v1 * dxLambda \
          - D_area * q1 * inner(tau, grad(v1)) * u1 * dxLambda \
          + D_area*q1*u1_up*jump(v1)*dSLambda + D_area * q1 * u1 * v1 * dsLambda(2)


      L0 = inner(0, v3) * dxOmega + inv_dt * (inner(uh3d_prev, v3) * dxOmega)
      L1 = inner(0, v1) * dxLambda \
          + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
          + D_area * q1 * cinlet * v1 * dsLambda(1)



      a = [[a00, a01], [a10, a11]]
      L = [L0, L1]

      #bc3d = DirichletBC(V3, u3_exact, "on_boundary")
      bc3d = DirichletBC(V3, 0.0,"on_boundary")
      W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

      A, b = map(ii_assemble, (a, L))
      A, b = apply_bc(A, b, W_bcs)
      A, b = map(ii_convert, (A, b))

      solver = PETScKrylovSolver()
      solver.set_operators(A, A)
      ksp = solver.ksp()

      opts = PETSc.Options()
      opts.setValue('ksp_type', 'gmres')
      opts.setValue('ksp_norm_type', 'unpreconditioned')
      opts.setValue('ksp_atol', 1E-14)
      opts.setValue('ksp_rtol', 1E-30)
      opts.setValue('ksp_monitor_true_residual', None)
      opts.setValue('pc_type', 'hypre')
      ksp.setFromOptions()

      x = b.copy()
      solver.solve(A,x,b)

      wh = ii_Function(W)
      wh.vector()[:] = x
      uh3d, uh1d = wh

      uh3d_prev.assign(uh3d)
      uh1d_prev.assign(uh1d)

      avg_uh3d = Average(uh3d, Lambda, cylinder)
      #avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

      # To compare without exact solution, you must project to a continuous space

      # === Project solution to continuous function space ===
      #V3c  = FunctionSpace(mesh_3d, "CG", 1)
      V1c  = FunctionSpace(Lambda, "CG", 1)

      wc = project(uh1d, V = V1c) #have to manually switch between 3d and 1d

      wn = wc


      # === Store Model Predictions ===
      vtkfile = File('3d_Prediction.pvd')
      vtkfile << uh3d

      vtkfile = File('1d_Prediction.pvd')
      vtkfile << uh1d

  return(uh3d)



In [9]:
#@title Convergence test with no known solution

kappa = 1.0
T = 1.0

# === Run different meshes ===
w4 = FEM3D_dG1D(kappa = kappa, mesh_size = [0], T = T)
w8 = FEM3D_dG1D(kappa = kappa, mesh_size = [1], T = T)
w16 = FEM3D_dG1D(kappa = kappa, mesh_size = [2], T = T)
w32 = FEM3D_dG1D(kappa = kappa, mesh_size = [3], T = T)
w64 = FEM3D_dG1D(kappa = kappa, mesh_size = [4], T = T) # assume finest mesh to be "true" solution
#w128 = FEM3D_dG1D(kappa = kappa, mesh_size = [5], T = T) # may need to use this as our finest to see convergence rate of w64



[INFO] Time step 1/10, t = 0.1


Averaging over 4 cells: 100%|██████████| 4/4 [00:00<00:00, 424.14it/s]

  0 KSP unpreconditioned resid norm 3.926990816987e-02 true resid norm 3.926990816987e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 7.686494691801e-05 true resid norm 7.686494691903e-05 ||r(i)||/||b|| 1.957349800426e-03
  2 KSP unpreconditioned resid norm 3.825956634036e-07 true resid norm 3.825956634051e-07 ||r(i)||/||b|| 9.742718565832e-06
  3 KSP unpreconditioned resid norm 8.946952424379e-10 true resid norm 8.946952093182e-10 ||r(i)||/||b|| 2.278322641978e-08
  4 KSP unpreconditioned resid norm 1.076205794798e-11 true resid norm 1.076205219230e-11 ||r(i)||/||b|| 2.740534086748e-10
  5 KSP unpreconditioned resid norm 3.096726986993e-16 true resid norm 5.628359519047e-16 ||r(i)||/||b|| 1.433249982328e-14
[INFO] Time step 2/10, t = 0.2
  0 KSP unpreconditioned resid norm 4.494645787582e-02 true resid norm 4.494645787582e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.825391261715e-05 true resid norm 2.825391261724e-05 ||r(i)||/||

  0 KSP unpreconditioned resid norm 4.602423118612e-02 true resid norm 4.602423118612e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.637311557148e-05 true resid norm 1.637311557150e-05 ||r(i)||/||b|| 3.557498984674e-04
  2 KSP unpreconditioned resid norm 6.068340371089e-08 true resid norm 6.068340371956e-08 ||r(i)||/||b|| 1.318509883938e-06
  3 KSP unpreconditioned resid norm 4.977437389364e-10 true resid norm 4.977437337062e-10 ||r(i)||/||b|| 1.081481908287e-08
  4 KSP unpreconditioned resid norm 3.906932870320e-13 true resid norm 3.906969118518e-13 ||r(i)||/||b|| 8.488939451738e-12
  5 KSP unpreconditioned resid norm 3.105852241312e-16 true resid norm 3.164701469570e-16 ||r(i)||/||b|| 6.876163681631e-15
[INFO] Time step 7/10, t = 0.7
  0 KSP unpreconditioned resid norm 4.602512218643e-02 true resid norm 4.602512218643e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.638347373837e-05 true resid norm 1.638347373837e-05 ||r(i)||/||

Averaging over 8 cells: 100%|██████████| 8/8 [00:00<00:00, 515.56it/s]

...

[INFO] Time step 2/10, t = 0.2
  0 KSP unpreconditioned resid norm 4.161832153153e-02 true resid norm 4.161832153153e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 4.440536538408e-04 true resid norm 4.440536538371e-04 ||r(i)||/||b|| 1.066966752854e-02
  2 KSP unpreconditioned resid norm 5.001834974020e-06 true resid norm 5.001834987155e-06 ||r(i)||/||b|| 1.201834865773e-04
  3 KSP unpreconditioned resid norm 6.156722729841e-08 true resid norm 6.156722259495e-08 ||r(i)||/||b|| 1.479329783838e-06
  4 KSP unpreconditioned resid norm 6.493061339638e-10 true resid norm 6.493056030128e-10 ||r(i)||/||b|| 1.560143655772e-08
  5 KSP unpreconditioned resid norm 5.479866678631e-12 true resid norm 5.474966511484e-12 ||r(i)||/||b|| 1.315518336639e-10
  6 KSP unpreconditioned resid norm 5.211881453494e-14 true resid norm 5.768522760908e-14 ||r(i)||/||b|| 1.386053677474e-12
  7 KSP unpreconditioned resid norm 5.047715403216e-16 true resid norm 4.373505610877e-14 ||r(i)||/||

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 510.86it/s]


  0 KSP unpreconditioned resid norm 3.926990816987e-02 true resid norm 3.926990816987e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.612214797772e-04 true resid norm 2.612214797477e-04 ||r(i)||/||b|| 6.651950358981e-03
  2 KSP unpreconditioned resid norm 2.633528524119e-06 true resid norm 2.633528584558e-06 ||r(i)||/||b|| 6.706225472100e-05
  3 KSP unpreconditioned resid norm 2.625625610639e-08 true resid norm 2.625622575807e-08 ||r(i)||/||b|| 6.686092986133e-07
  4 KSP unpreconditioned resid norm 3.263095483599e-10 true resid norm 3.263322763635e-10 ||r(i)||/||b|| 8.309983179789e-09
  5 KSP unpreconditioned resid norm 3.205101422545e-12 true resid norm 3.192277092880e-12 ||r(i)||/||b|| 8.129066864815e-11
  6 KSP unpreconditioned resid norm 3.906275565969e-14 true resid norm 1.927839383786e-13 ||r(i)||/||b|| 4.909202678669e-12
  7 KSP unpreconditioned resid norm 6.035739377756e-16 true resid norm 1.408334779247e-13 ||r(i)||/||b|| 3.586295066326e-12
[INFO] T

Averaging over 64 cells: 100%|██████████| 64/64 [00:00<00:00, 507.33it/s]


  0 KSP unpreconditioned resid norm 3.926990816987e-02 true resid norm 3.926990816987e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.366073344219e-04 true resid norm 2.366073344996e-04 ||r(i)||/||b|| 6.025156297185e-03
  2 KSP unpreconditioned resid norm 3.468226975881e-06 true resid norm 3.468226988166e-06 ||r(i)||/||b|| 8.831767502900e-05
  3 KSP unpreconditioned resid norm 3.635449900530e-08 true resid norm 3.635443643193e-08 ||r(i)||/||b|| 9.257581218338e-07
  4 KSP unpreconditioned resid norm 3.333507469367e-10 true resid norm 3.333579967903e-10 ||r(i)||/||b|| 8.488891681342e-09
  5 KSP unpreconditioned resid norm 5.577757966259e-12 true resid norm 5.559238365138e-12 ||r(i)||/||b|| 1.415648425020e-10
  6 KSP unpreconditioned resid norm 8.125389088772e-14 true resid norm 4.151050573667e-13 ||r(i)||/||b|| 1.057056348518e-11
  7 KSP unpreconditioned resid norm 1.301431607450e-15 true resid norm 4.675228388361e-13 ||r(i)||/||b|| 1.190537132946e-11
[INFO] T

In [10]:
#@title Compute errors and rates
# === Create mesh for your "true" solution ===
mesh64 = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),64,64,64)
zero = Constant(0.0)
V = FunctionSpace(mesh64, "CG", 1)
zeromesh = interpolate(zero, V)

# === w8 errors ===
norm_error = errornorm(w4, w64, 'L2')
norm_ref   = norm(w64, 'L2')
relative_error_w4 =  norm_error/norm_ref
print("The relative error between w4 and w64 is", relative_error_w4)

# === w8 errors ===
norm_error = errornorm(w8, w64, 'L2')
relative_error_w8 =  norm_error/norm_ref
print("The relative error between w8 and w64 is", relative_error_w8)

# === w16 errors ===
norm_error = errornorm(w16, w64, 'L2')
relative_error_w16 =  norm_error/norm_ref
print("The relative error between w16 and w64 is", relative_error_w16)

# === w32 errors ===
norm_error = errornorm(w32, w64, 'L2')
relative_error_w32 =  norm_error/norm_ref
print("The relative error between w32 and w64 is", relative_error_w32)

print("Convergence rate for n=8 is", np.log(relative_error_w4/relative_error_w8)/np.log(2))
print("Convergence rate for n=16 is", np.log(relative_error_w8/relative_error_w16)/np.log(2))
print("Convergence rate for n=32 is", np.log(relative_error_w16/relative_error_w32)/np.log(2))


Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_0ffb7e52403eaee1b83d42c8e168d762aaa1bdb1

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      0
  Arguments:                 '()'
  Number of coefficients:    1
  Coefficients:              '[f_14712]'
  Unique elements:           'DG4(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'DG4(?,?), Vector<3 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 8
INFO:FFC:  quadrature_degree: 8


           Consider using the option 'quadrature_degree' to reduce the number of points


INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0170491 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 0 elements
INFO:FFC:  Computing representation of 0 dofmaps
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing uflacs representation
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DE

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_a63ab5bd0d0e5b2b940cdcfa62c1e5b6545c155c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00130868 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.00983119 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representat

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_69bafdc5a3894735f92630c7ba282d4a8754c1ec

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      0
  Arguments:                 '()'
  Number of coefficients:    1
  Coefficients:              '[f_14668]'
  Unique elements:           'CG1(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'CG1(?,?), Vector<3 x CG1(?,?)>'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 2
INFO:FFC:  quadrature_degree: 2
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00938153 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-----------------------------

The relative error between w4 and w64 is 0.5220371249799569
The relative error between w8 and w64 is 0.22030837878706783
The relative error between w16 and w64 is 0.08907326909419298
The relative error between w32 and w64 is 0.02501725690274168
Convergence rate for n=8 is 1.2446280437743729
Convergence rate for n=16 is 1.3064599160409542
Convergence rate for n=32 is 1.8320689343723162
